In [2]:
from functools import partial
import json
import pandas as pd
import requests


import shapely
from shapely.geometry import Point
from shapely.ops import transform
import pyproj

In [3]:
import matplotlib.pyplot as plt
import numpy as np
#import pandas as pd

with open("./gb1900_gazetteer_complete_july_2018.csv", encoding='UTF-16') as f:
    df = pd.read_csv(f)

/anaconda3/envs/py3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,pin_id,final_text,nation,local_authority,parish,osgb_east,osgb_north,latitude,longitude,notes
0,52b34d8b695fe90005004e1e,F. P.,Wales,Powys,Llansilin,320836.712742,327820.182715,52.842050,-3.176744,NaN
1,5800a6b92c66dcab3d061796,Parly. & Munl Boro. By.,England,City of London,NaN,531794.825962,180705.741898,51.509918,-0.102246,NaN
2,5800a6782c66dcab3d061786,S. Ps.,England,City of London,NaN,531736.217116,180725.027730,51.510105,-0.103083,NaN
3,57f684f42c66dcab3d01c0dd,Southwark Bridge Stairs,England,City of London,NaN,532199.584123,180696.934434,51.509744,-0.096420,NaN
4,57f685002c66dcab3d01c0e9,St. Paul's Pier,England,City of London,NaN,531987.486097,180745.664556,51.510232,-0.099456,NaN


In [6]:
df[df["final_text"] == 'The Magnet (Reading Room)']

,pin_id,final_text,nation,local_authority,parish,osgb_east,osgb_north,latitude,longitude,notes
28157,57edb5ff2c66dca3220042dd,The Magnet (Reading Room),England,Rugby,Rugby (Un-parished),448311.022696,273577.247439,52.358134,-1.292042,NaN


In [8]:
search_url = 'http://51.140.84.209:9200'

response = requests.get('{}/_cat/indices?format=json'.format(search_url))
indices = {}
for i in response.json():
    index_name = i['index']
    if not index_name.startswith('.'):
        indices[index_name[:2]] = index_name
indices

ConnectionError: HTTPConnectionPool(host='51.140.84.209', port=9200): Max retries exceeded with url: /_cat/indices?format=json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11fff67b8>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [ ]:
azure_container_name = 'maps'
map_folders = {
    '01': '01_OS_Town_Plans_London_1890s_GeoTIFFs',
    '02': '02_OS_Town_Plans_England_1840s_1890s_GeoTIFFs',
    '03': '03_OS_25_Inch_Lancs_Dorset_1900s_1940s_GeoTIFFs',
    '04': '04_OS_Six_Inch_Scotland_1840s_1880s_GeoTIFFs',
    '05': '05_OS_Six_Inch_Lancs_Dorset_1880s_1890s_TIFFs',
    '06': '06_OS_One_Inch_Scotland_Lancs_Dorset_London_1850s_1900s_TIFFs',
    '07': '07_OS_One_Inch_Scotland_Lancs_Dorset_London_1890s_1900s_GeoTIFFs'
}


#limit to 05 folder?

In [ ]:
def create_circle(centre, radius):
    proj_4326 = pyproj.Proj(init='EPSG:4326')
    proj_32633 = pyproj.Proj(init='EPSG:32633')
    transform_to_32633 = partial(pyproj.transform, proj_4326, proj_32633)
    transform_to_4326 = partial(pyproj.transform, proj_32633, proj_4326)
    proj_circle = transform(transform_to_32633, centre).buffer(radius, 3)
    circle = transform(transform_to_4326, proj_circle)
    return circle

In [ ]:
def create_geoquery(geojson):
    return {
       "query":{  
          "bool":{  
             "must": [  
                {  
                   "geo_shape": {  
                      "location": {  
                         "shape": geojson,
                         "relation":"INTERSECTS"
                      }
                   }
                }
             ],
             "filter":[  
                {  
                   "match_all": {}
                }
             ],
             "should": [],
             "must_not": []
          }
       }
    }


In [ ]:
def find_maps(es_index, area):
    geojson = shapely.geometry.mapping(area)
    # we have to flip the lat-lon coordinates to comply with geojson
    flipped = [[coord[1], coord[0]] for coord in geojson['coordinates'][0]]
    geojson['coordinates'] = [flipped]
    geoquery = create_geoquery(geojson)
    response = requests.post('{}/{}/_search'.format(search_url, es_index), json=geoquery)
    #print('{}/{}/_search'.format(search_url, es_index))
    #print(geoquery)
    hits = response.json()['hits']
    return hits

In [ ]:
def create_dataframe(hits):
    return pd.concat(map(lambda h: pd.DataFrame.from_dict(h)['_source'].to_frame().T, hits['hits']))

In [ ]:
#for each pin_id, pull latitude, longitude from gb1900 df

#Example for Lancaster castle pin
coordinates = {
    '57edb5ff2c66dca3220042dd': Point(52.358134, -1.292042),
}

pin_area = create_circle(coordinates['57edb5ff2c66dca3220042dd'], 100)


#why none?

In [ ]:
coordinates

In [ ]:
# For gb1900, we only work with
# '05': '05_OS_Six_Inch_Lancs_Dorset_1880s_1890s_TIFFs',

# key for '05_OS_Six_Inch_Lancs_Dorset_1880s_1890s_TIFFs' map dataset
key = '05'
index = indices[key]

for key, index in indices.items():
    hits = find_maps(index, pin_area)
    num_hits = hits['total']['value']
    print('{}: {} hits'.format(index, num_hits))
    print('Azure folder: {}'.format(map_folders[key]))
    for hit in hits['hits']:
        print(hit['_source']['IMAGE'])
    print()

In [ ]:
hits

In [ ]:
#append sheet id from 05 folder to this URL format:
#https://maps.nls.uk/view/102343805

#add new column in gb1900 df and insert url